In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/Users/iluzaangirova/Desktop/Bert/modeling.py" .
!cp "/Users/iluzaangirova/Desktop/Bert/optimization.py" .
!cp "/Users/iluzaangirova/Desktop/Bert/tokenization.py" .

In [4]:
import tokenization 
import torch
import sys 
import time
import numpy as np
import pandas as pd
from IPython.display import clear_output
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from modeling import BertConfig, BertForSequenceClassification
from optimization import BERTAdam
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score, matthews_corrcoef

import matplotlib.pyplot as plt
import tkinter as tk

In [6]:
df = pd.read_csv('/Users/iluzaangirova/Desktop/Bert/dataset_5.csv', encoding='utf8')
df = df[['topics', 'title']]
print(df.shape)
df.head()

(33068, 2)


,topics,title
0,sport,Суд американского города Торранс приступит к р...
1,sport,Состояние здоровья знаменитого канадского хокк...
2,sport,"Финский ""Йокерит"" потеряет преимущество своей ..."
3,sport,"""Сакраменто"" проиграл ""Мемфису"" в матче чемпио..."
4,sport,"ФК ""Валенсия"": сделаем все, чтобы найти фаната..."


In [7]:
maps = pd.factorize(df.topics)[1]
print(maps)
df['topics'] = pd.factorize(df.topics)[0]
print(df.head())
df['title'] = df['title'].astype('str')
for c in df:
    if df[c].dtype == 'object':
        print('Max длина предложения %s: %s\n' %  (c, df[c].map(len).max()))

Index(['sport', 'society', 'auto', 'business', 'economic'], dtype='object')
   topics                                              title
0       0  Суд американского города Торранс приступит к р...
1       0  Состояние здоровья знаменитого канадского хокк...
2       0  Финский "Йокерит" потеряет преимущество своей ...
3       0  "Сакраменто" проиграл "Мемфису" в матче чемпио...
4       0  ФК "Валенсия": сделаем все, чтобы найти фаната...
Max длина предложения title: 162



# Train

In [8]:
sentences = df['title'].values
labels = df['topics'].to_list()
assert len(sentences) == len(labels)

In [9]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, label):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.label = label

In [10]:
train_sentences, test_sentences, train_gt, test_gt = train_test_split(sentences, labels, shuffle=True,test_size=0.3, random_state=42)
assert len(set(train_gt)) == len(set(test_gt))
num_classes = len(set(train_gt))

In [11]:
kf = KFold(n_splits=3, shuffle=True)

In [12]:
def preprocessing(sentences, labels, tokenizer, max_len):
    features = []
    for i,sentence in enumerate(sentences):
        
        tokens_a = tokenizer.tokenize(sentence)
        
        if len(tokens_a) > max_len - 2:
            tokens_a = tokens_a[0:(max_len - 2)]

        tokens = []
        tokens.append("[CLS]")
        for token in tokens_a:
            tokens.append(token)
        tokens.append("[SEP]")

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_len:
            input_ids.append(0)
            input_mask.append(0)
            
        assert len(input_ids) == max_len
        assert len(input_mask) == max_len

        features.append(
                    InputFeatures(
                            input_ids=input_ids,
                            input_mask=input_mask,
                            label=[labels[i]]))
    
    return features

In [13]:
class Dataload(torch.utils.data.Dataset):
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):

        
        return torch.LongTensor(self.features[index].input_ids),\
               torch.LongTensor(self.features[index].input_mask),\
               torch.LongTensor(self.features[index].label)

In [14]:
tokenizer = tokenization.FullTokenizer(vocab_file='./gdrive/My Drive/Transformer/vocab.txt', do_lower_case=False)

FileNotFoundError: [Errno 2] No such file or directory: './gdrive/My Drive/Transformer/vocab.txt'

In [22]:
for fold, (train_index, test_index) in enumerate(kf.split(sentences, labels)):
    print("TRAIN:", len(train_index), "TEST:", len(test_index), fold)
    train_sentences, test_sentences = sentences[train_index], sentences[test_index]
    train_gt, test_gt = labels[train_index], labels[test_index]
    
    features = preprocessing(train_sentences, train_gt, tokenizer, 512)
    dataset_train = Dataload(features)
    train_dataloader = torch.utils.data.DataLoader(dataset_train,batch_size = 8, shuffle=True,\
                                                  num_workers=6, pin_memory=True)
    features = preprocessing(test_sentences, test_gt, tokenizer, 512)
    dataset_test = Dataload(features)
    test_dataloader = torch.utils.data.DataLoader(dataset_test,batch_size = 1, shuffle=False,\
                                                  num_workers=6, pin_memory=True)
    
    print(len(train_dataloader))
    print(len(test_dataloader))

    device = 'cuda'
    bert_config = BertConfig.from_json_file('./gdrive/My Drive/Transformer/bert_config.json')
    model = BertForSequenceClassification(bert_config, num_classes)
    model.bert.load_state_dict(torch.load('./gdrive/My Drive/Transformer/pytorch_model.bin'\
                                          , map_location='cpu'))
    model.to(device)
    model = torch.nn.DataParallel(model)

    num_epoch = 5
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
        ]
    num_train_steps = len(train_dataloader) * num_epoch
    optimizer = BERTAdam(optimizer_parameters,
                        lr=5e-5,
                        warmup=0.1,
                        t_total=num_train_steps)
    
    f = open('log'+str(fold)+'.txt', 'w')
    f.close()


    train_loss_set = []

    batch_iterator = iter(train_dataloader)

    total_step = 0
    model.train()
    train_loss = 0

    while total_step<5*len(train_dataloader):

        total_step += 1
        try:
            batch = next(batch_iterator)
        except:
            batch_iterator = iter(data_loader)
            batch = next(batch_iterator)

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()

        loss, logits = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels.squeeze(1), token_type_ids=None)
        
        train_loss_set.append(loss.mean().item())

        loss.mean().backward()

        optimizer.step()

        train_loss += loss.mean().item()

        clear_output(True)
        plt.plot(train_loss_set)
        plt.title("Training loss")
        plt.xlabel("Batch")
        plt.ylabel("Loss")
        plt.show()

        if total_step%1000 == 0:
            print("Mean loss: {0:.5f}".format(train_loss / len(train_dataloader)))
            with open('log'+str(fold)+'.txt', 'a') as f:
                f.write("Mean loss: {0:.5f}\n".format(train_loss / len(train_dataloader)))
            torch.save(model.state_dict(), './gdrive/My Drive/weights'+str(fold)+'_'+str(total_step)+'.pth')
            train_loss = 0
            model.eval()
            valid_preds, valid_labels = [], []

            for batch in test_dataloader:   

                batch = tuple(t.to(device) for t in batch)

                b_input_ids, b_input_mask, b_labels = batch

                with torch.no_grad():
                    logits = model(b_input_ids, attention_mask=b_input_mask, token_type_ids=None)

                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.squeeze(1).to('cpu').numpy()
                batch_preds = np.argmax(logits, axis=1)
                batch_labels = np.hstack(label_ids)

                valid_preds.extend(batch_preds)
                valid_labels.extend(batch_labels)

            with open('log'+str(fold)+'.txt', 'a') as f:
                f.write("Accuracy: {0:.2f}%".format(
                  accuracy_score(valid_labels, valid_preds) * 100
                ))
                f.write("Matthews: {0:.2f}%".format(
                  matthews_corrcoef(valid_labels, valid_preds) * 100
                ))
            model.train()

TRAIN: 22045 TEST: 11023 0


TypeError: only integer scalar arrays can be converted to a scalar index

# Predict

In [18]:
device = 'cpu'
bert_config = BertConfig.from_json_file('/Users/iluzaangirova/Desktop/Bert/train_predict/bert_config.json')
model = BertForSequenceClassification(bert_config, 5)
model.to(device)
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load('/Users/iluzaangirova/Desktop/Bert/weights5000.pth'\
                                      , map_location='cpu'))

<All keys matched successfully>

In [19]:
tokenizer = tokenization.FullTokenizer(vocab_file='/Users/iluzaangirova/Desktop/Bert/vocab.txt', do_lower_case=False)

In [20]:
def predict(sentence, tokenizer, max_len):
#     print(sentence)
    tokens_a = tokenizer.tokenize(sentence)
    if len(tokens_a) > max_len - 2:
        tokens_a = tokens_a[0:(max_len - 2)]

    tokens = []
    tokens.append("[CLS]")
    for token in tokens_a:
        tokens.append(token)
    tokens.append("[SEP]")
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)
    while len(input_ids) < max_len:
        input_ids.append(0)
        input_mask.append(0)
    input_ids = torch.LongTensor(input_ids).unsqueeze(0)
    input_mask = torch.LongTensor(input_mask).unsqueeze(0)
    a = time.time()

    logits = model(input_ids.cpu(), None, input_mask.cpu())
    logits = logits.squeeze(0)
    
    b = time.time()
    logits = F.softmax(logits, dim=-1)
    clas = logits.argmax().item()
    return clas

In [81]:
t = "В Молдавии приостановили действие поправок о полномочиях президента"
maps[predict(t, tokenizer, 512)]

'society'

In [27]:
maps

Index(['sport', 'society', 'auto', 'business', 'economic'], dtype='object')

In [21]:
window = tk.Tk()
window.title('Classifier')
label = tk.Label(text="Введите заголовок статьи")
entry = tk.Entry()
label.pack()
entry.pack()
title1 = entry.get()
def button():
    label = tk.Label(window, text=maps[predict(entry.get(), tokenizer, 512)])
    label.pack()
btn1 = tk.Button(window, text = 'классифицировать', command = button)
btn1.pack()
window.mainloop()